In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.io import read_image
from typing import List, Tuple
import os
import glob
import json

In [ ]:
class SoccerDataset(Dataset):
    def __init__(self, images: List[str], targets: List[int], type: str ):
        self.images = images
        self.targets = targets
        self.n_samples = len(targets)
        # Create a PyTorch trasnform that resizes the images and norn§alize it
        if type == 'train':
            self.transforms = v2.Compose([
                v2.Resize(size=(224, 224), antialias=True),
                v2.RandomHorizontalFlip(p=0.5),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        else:
            self.transforms = v2.Compose([
                v2.Resize(size=(224, 224), antialias=True),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = v2.Compose([
            v2.Resize(size=(224, 224), antialias=True),
            v2.RandomHorizontalFlip(p=0.5),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        unique_targets = set(self.targets)
        self.targets2idx = {l: i for i, l in enumerate(unique_targets)}
        self.idx2targets = {i: l for i, l in enumerate(unique_targets)}

    def __getitem__(self, index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        img = read_image(self.images[index])
        img = self.transforms(img)
        target = self.targets[index]
        target = torch.tensor(self.targets2idx[target], dtype=torch.long)
        return img, target
    
    def __len__(self) -> int:
        return self.n_samples

In [ ]:
dataset_path = "SoccerNet/jersey-2023/"
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')
train_images = glob.glob(os.path.join(train_path, 'images/**/*.jpg'), recursive=True)
test_images = glob.glob(os.path.join(test_path, 'images/**/*.jpg'), recursive=True)

In [ ]:
from ultralytics import YOLO
from torchvision.io import read_image
from torchvision.transforms.functional import resize
import numpy as np
import cv2


mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

train_gt = json.load(open(os.path.join(train_path, 'train_gt.json')))
train_targets = []
for id in os.listdir(os.path.join(train_path, 'images/')):
    try:
        int(id)
        train_targets.extend([train_gt[id]] * len(os.listdir(os.path.join(train_path, 'images/', id))))
    except Exception:
        print(f"Skipping {id}, succhiamelo")

In [ ]:
test_gt = json.load(open(os.path.join(test_path, 'test_gt.json')))
test_targets = []
for id in os.listdir(os.path.join(test_path, 'images/')):
    try:
        int(id)
        test_targets.extend([test_gt[id]] * len(os.listdir(os.path.join(test_path, 'images/', id))))
    except Exception:
        print(f"Skipping {id}, succhiamelo")

In [ ]:
import torch.utils.data as data

train_data = SoccerDataset(train_images, train_targets, 'train')
test_data = SoccerDataset(test_images, test_targets, 'test')

train_set_size = int(len(train_data) * 0.8)
valid_set_size = len(train_data) - train_set_size
seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(train_data, [train_set_size, valid_set_size], generator=seed)

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.utils import data

# Assuming you have a class_labels tensor with the class labels for each sample
# Replace this with your actual class labels
class_labels = torch.tensor(train_targets)

# Convert indices to integers
train_indices = [int(idx) for idx in train_set.indices]
valid_indices = [int(idx) for idx in valid_set.indices]

# Get class labels for each set
train_class_labels = class_labels[train_indices]
valid_class_labels = class_labels[valid_indices]
test_class_labels = torch.tensor(test_targets)

for label in train_class_labels:
    if label < 0:
        train_class_labels[train_class_labels == label] = 99

for label in valid_class_labels:
    if label < 0:
        valid_class_labels[valid_class_labels == label] = 99

for label in test_class_labels:
    if label < 0:
        test_class_labels[test_class_labels == label] = 99

# Count the occurrences of each class label in each set
train_class_counts = torch.bincount(train_class_labels)
valid_class_counts = torch.bincount(valid_class_labels)
test_class_counts = torch.bincount(test_class_labels)



In [ ]:
import numpy as np

# Funzione per dividere le classi in gruppi di n
def split_classes(class_counts, n):
    return [class_counts[i:i+n] for i in range(0, len(class_counts), n)]

# Imposta la spaziatura tra le classi sull'asse x
bar_spacing = 0.4  # Regola questo valore secondo le tue preferenze

# Calcola le posizioni delle classi per ciascun set
train_positions = np.arange(len(train_class_counts))
valid_positions = np.arange(len(valid_class_counts))
test_positions = np.arange(len(test_class_counts))

# Divide le classi in 4 gruppi per ciascun set
train_class_groups = split_classes(train_class_counts, len(train_class_counts)//4)
valid_class_groups = split_classes(valid_class_counts, len(valid_class_counts)//4)
test_class_groups = split_classes(test_class_counts, len(test_class_counts)//4)

In [ ]:
total_train_sum = sum(train_class_counts)
for i, class_group in enumerate(train_class_groups):
    normalized_values = [value / total_train_sum for value in class_group]
    tick_labels = range(i * len(normalized_values), (i + 1) * len(normalized_values))
    
    plt.figure(figsize=(10, 5))
    bars = plt.bar(train_positions[:len(normalized_values)], class_group, tick_label=tick_labels)
    plt.title(f'Class Distribution in Train Set - Group {i+1}')
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')

    for bar, value in zip(bars, normalized_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2%}', 
                 ha='center', va='bottom', fontsize=6)  # Regola la dimensione del font qui

    plt.show()

In [ ]:
total_valid_sum = sum(valid_class_counts)
for i, class_group in enumerate(valid_class_groups):
    normalized_values = [value / total_valid_sum for value in class_group]
    tick_labels = range(i * len(normalized_values), (i + 1) * len(normalized_values))
    
    plt.figure(figsize=(10, 5))
    bars = plt.bar(valid_positions[:len(normalized_values)], class_group, tick_label=tick_labels)
    plt.title(f'Class Distribution in Validation Set - Group {i+1}')
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')

    for bar, value in zip(bars, normalized_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2%}', 
                 ha='center', va='bottom', fontsize=6)

    plt.show()


In [ ]:
total_test_sum = sum(test_class_counts)
for i, class_group in enumerate(test_class_groups):
    normalized_values = [value / total_test_sum for value in class_group]
    tick_labels = range(i * len(normalized_values), (i + 1) * len(normalized_values))
    
    plt.figure(figsize=(10, 5))
    bars = plt.bar(test_positions[:len(normalized_values)], class_group, tick_label=tick_labels)
    plt.title(f'Class Distribution in Test Set - Group {i+1}')
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')

    for bar, value in zip(bars, normalized_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2%}', 
                 ha='center', va='bottom', fontsize=6)

    plt.show()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import v2
from torchvision.io import read_image
from typing import List, Tuple
import os
import glob
import json

In [ ]:
class SoccerDataset(Dataset):
    def __init__(self, images: List[str], targets: List[int], type: str ):
        self.images = images
        self.targets = targets
        self.n_samples = len(targets)
        # Create a PyTorch trasnform that resizes the images and norn§alize it
        if type == 'train':
            self.transforms = v2.Compose([
                v2.Resize(size=(224, 224), antialias=True),
                v2.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
                v2.RandomGrayscale(0.5),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        else:
            self.transforms = v2.Compose([
                v2.Resize(size=(224, 224), antialias=True),
                v2.ToDtype(torch.float32, scale=True),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        unique_targets = set(self.targets)
        self.targets2idx = {l: i for i, l in enumerate(unique_targets)}
        self.idx2targets = {i: l for i, l in enumerate(unique_targets)}

    def __getitem__(self, index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        img = read_image(self.images[index]) / 255.0
        img = self.transforms(img)
        target = self.targets[index]
        target = torch.tensor(self.targets2idx[target], dtype=torch.long)
        return img, target
    
    def __len__(self) -> int:
        return self.n_samples
    
class SoccerNet(torch.nn.Module):
    def __init__(self, num_classes: int ):
        super().__init__()
        self.model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [ ]:
from typing import Any
import lightning as L
import torchmetrics

class LitSoccerNet (L.LightningModule):
    def __init__(self, net: SoccerNet, num_classes: int):
        super().__init__()
        self.net = net
        self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)
        self.valid_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        logits = self.net(images)
        loss = torch.nn.functional.cross_entropy(logits, targets)
        accuracy = self.train_acc(logits, targets)
        self.log("train_loss", loss)
        self.log("train_accuracy", accuracy, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=3e-4)
        return optimizer

    def test_step(self, batch, batch_idx):
        images, targets = batch
        logits = self.net(images)
        loss = torch.nn.functional.cross_entropy(logits, targets)
        self.log("test_loss", loss)
        accuracy = self.test_acc(logits, targets)
        self.log("test_accuracy", accuracy, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        logits = self.net(images)
        val_loss = torch.nn.functional.cross_entropy(logits, targets)
        accuracy = self.valid_acc(logits, targets)
        self.log("val_loss", val_loss)
        self.log("val_accuracy", accuracy, on_epoch=True, on_step=True)

In [ ]:
import lightning as L

class SoccerNetDataModule (L.LightningDataModule):
    def __init__(self, data_dir: str, batch_size: int):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def prepare_data(self):
        self.train_path = os.path.join(self.data_dir, 'train')
        self.test_path = os.path.join(self.data_dir, 'test')

        self.train_images = glob.glob(os.path.join(self.train_path, 'images/**/*.jpg'), recursive=True)
        self.test_images = glob.glob(os.path.join(self.test_path, 'images/**/*.jpg'), recursive=True)

        train_gt = json.load(open(os.path.join(self.train_path, 'train_gt.json')))
        self.train_targets = []
        for id in os.listdir(os.path.join(self.train_path, 'images/')):
            try:
                int(id)
                self.train_targets.extend([train_gt[id]] * len(os.listdir(os.path.join(self.train_path, 'images/', id))))
            except Exception:
                print(f"Skipping {id}")

        test_gt = json.load(open(os.path.join(self.test_path, 'test_gt.json')))
        self.test_targets = []
        for id in os.listdir(os.path.join(self.test_path, 'images/')):
            try:
                int(id)
                self.test_targets.extend([test_gt[id]] * len(os.listdir(os.path.join(self.test_path, 'images/', id))))
            except Exception:
                print(f"Skipping {id}")
        
    
    def setup(self, stage: str):
        if stage == "fit" or stage is None:
            train_data = SoccerDataset(self.train_images, self.train_targets, type='train')
            self.idx2targets = train_data.idx2targets
            self.targets2idx = train_data.targets2idx
            train_set_size = int(len(train_data) * 0.8)
            valid_set_size = len(train_data) - train_set_size
            seed = torch.Generator().manual_seed(42)
            train_set, valid_set = random_split(train_data, [train_set_size, valid_set_size], generator=seed)
            self.train_set = train_set
            self.valid_set = valid_set
        
        if stage == "test" or stage is None:
            test_data = SoccerDataset(self.test_images, self.test_targets, type='test')
            self.test_set = test_data

        if stage == "predict" or stage is None:
            pass

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.valid_set, batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    
    def predict_dataloader(self):
        pass

In [ ]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

DATASET_PATH = "SoccerNet/jersey-2023/"
NUM_CLASSES = 45 #len(train_data.targets2idx)
BATCH_SIZE = 32

net = SoccerNet(num_classes=NUM_CLASSES)
autoSoccerNet = LitSoccerNet(net, NUM_CLASSES)
data_module = SoccerNetDataModule(DATASET_PATH, BATCH_SIZE)


checkpoint_callback = ModelCheckpoint(
    monitor='train_accuracy',
    filename='SoccerNet-{val_accuracy:.2f}-{val_loss:.2f}',
    verbose=True,
    dirpath='checkpointsV3/',
    every_n_train_steps=3000,
    enable_version_counter=True,
    save_top_k=-1
    )

trainer = L.Trainer(
    max_epochs=3,
    enable_checkpointing=True, 
    callbacks=[checkpoint_callback],
    )


In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

trainer.fit(autoSoccerNet, data_module)

In [ ]:
import numpy as np
import torch 
import torch.nn.functional as F
from torchvision import transforms
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

checkpoint = 'checkpoints/SoccerNet-train_accuracy=1.00-train_loss=0.02.ckpt'
autoSoccerNet = LitSoccerNet.load_from_checkpoint(checkpoint, net=net, num_classes=NUM_CLASSES,strict=False)

classifier = autoSoccerNet.net
classifier.eval()

# Move the classifier to the same device as your input data
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = classifier.to(device)


image = load_img('test.jpg')
image = img_to_array(image) / 255.0
torch_image = torch.from_numpy(image)
torch_image = torch_image.unsqueeze(0).permute(0,3,1,2)
transform = transforms.Compose([
    v2.Resize(size=(224, 224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.Grayscale(num_output_channels=3)
])

input_img = transform(torch_image)

preds = classifier(input_img)
print(preds.softmax(-1).argmax(1))
print(f"Most likely class: {data_module.idx2targets[torch.argmax(preds).item()]}")

plt.imshow(input_img.squeeze().permute(1,2,0))

In [ ]:
# TEST THE MODEL PERFORMANCE ON THE TEST SET WITH 3 IMAGES PER FOLDER, PRINTING THE NUMBER OF CORRECTLY IDENTIFIED JERSEYS


import numpy as np
import torch 
import torch.nn.functional as F
from torchvision import transforms
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from tqdm.auto import  tqdm

MODEL_CHECKPOINT = 'checkpointsV3/SoccerNet-train_accuracy=1.00-train_loss=0.01.ckpt'
autoSoccerNet = LitSoccerNet.load_from_checkpoint(MODEL_CHECKPOINT, net=net, num_classes=NUM_CLASSES,strict=False)

classifier = autoSoccerNet.net
classifier.eval()

# Move the classifier to the same device as your input data
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = classifier.to('cpu')
test_path = 'SoccerNet/jersey-2023/test/'
test_gt = json.load(open(os.path.join(test_path, 'test_gt.json')))
total_correct = 0
total_images = 0

for id in tqdm(os.listdir(os.path.join(test_path, 'images/')), desc= 'Testiing ...'):
    counter = 0
    correct = 0
    for image in os.listdir(os.path.join(test_path, 'images/', id)):
        image_path = os.path.join(test_path, 'images/', id, image)
        image = load_img(image_path)
        image = img_to_array(image) / 255.0
        torch_image = torch.from_numpy(image)
        torch_image = torch_image.unsqueeze(0).permute(0,3,1,2)
        transform = transforms.Compose([
            v2.Resize(size=(224, 224), antialias=True),
            v2.ToDtype(torch.float32, scale=True),
            #v2.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            v2.Grayscale(3),
            
        ])

        input_img = transform(torch_image)
        input_img.to(device)

        preds = classifier(input_img)
        predicted_class =  data_module.idx2targets[torch.argmax(preds).item()]
        real_class = test_gt[id]
        print(f'Real class {real_class}, Predcited class {predicted_class}')

        if predicted_class == real_class:
            correct += 1
            total_correct += 1

        counter += 1
        total_images += 1
        if counter == 10:
            print(f'Number correctly identified for {id} with real class {real_class} : {correct}/10')
            print('------------------------------------------------------------------------------------------------')
            break


print(f'Total corrected predictions: {total_correct}/{total_images}')